In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
pd.set_option('display.max_columns', None)
import tensorflow as tf
import xml.etree.ElementTree as ET

In [2]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score,ndcg_score,mean_squared_error

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Flatten, Dropout, Embedding,BatchNormalization,Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [4]:
data_path = 'processed_data/'

In [5]:
## ratings ##
df_ratings_score = pd.read_csv(data_path + 'Ratings_Reviews.csv')
df_ratings_score = df_ratings_score[['PERSON_ID','SurveyAnswerScore', 'RecommendStar', 'ProviderCode']]
df_ratings_score=df_ratings_score.groupby(['PERSON_ID','ProviderCode']).agg({'SurveyAnswerScore' : 'mean','RecommendStar':'mean'}).reset_index()
df_ratings_score['ratings'] = df_ratings_score['SurveyAnswerScore'] * 0.5 + df_ratings_score['RecommendStar'] *0.5
df_ratings_score['ratings']=df_ratings_score['ratings'].apply(lambda x : np.round(x,2))
#df_ratings_score = df_ratings_score.drop(['RecommendStar','SurveyAnswerScore'],axis=1)
df_ratings_score.columns

Index(['PERSON_ID', 'ProviderCode', 'SurveyAnswerScore', 'RecommendStar',
       'ratings'],
      dtype='object')

In [6]:
##facility##
df_facility = pd.read_csv(data_path + 'Provider/facility.csv')
def extract_zip_code(address):
    # Regular expression to match 5-digit zip code
    match = re.search(r'\b\d{5}\b', address)
    if match:
        return match.group()
    return None

df_facility = df_facility[['FACILITYCODE','FACILITYNAME','FACILITYTYPECODE','ADDRESSXML']]
df_facility['zip_code'] = df_facility['ADDRESSXML'].apply(lambda x : extract_zip_code(x))
df_facility=df_facility.drop(['ADDRESSXML'],axis=1)
df_facility.columns

Index(['FACILITYCODE', 'FACILITYNAME', 'FACILITYTYPECODE', 'zip_code'], dtype='object')

In [7]:
#speciality
df_speciality = pd.read_csv(data_path + 'Provider/speciality.csv')
df_speciality = df_speciality[['SpecialtyCode','SpecialtyDescription','SpecialistDescription']]
df_speciality.columns

Index(['SpecialtyCode', 'SpecialtyDescription', 'SpecialistDescription'], dtype='object')

In [8]:
#provider facility
df_provider_facility = pd.read_csv(data_path + 'Provider/ProviderToFacility.csv')
df_provider_facility = df_provider_facility[['PROVIDERCODE','FACILITYCODE']]
df_provider_facility.columns

Index(['PROVIDERCODE', 'FACILITYCODE'], dtype='object')

In [9]:
#provider spciality
df_provider_speciality = pd.read_csv(data_path + 'Provider/ProviderToSpecailty.csv')
df_provider_speciality = df_provider_speciality[['PROVIDERCODE','SPECIALTYCODE','SPECIALTYRANKCALCULATED']]
df_provider_speciality.columns

Index(['PROVIDERCODE', 'SPECIALTYCODE', 'SPECIALTYRANKCALCULATED'], dtype='object')

In [10]:
## create segments 
# Merge the dataframes to create the desired dataframe
df_merged = df_provider_facility.merge(df_facility, on='FACILITYCODE').merge(df_provider_speciality, on='PROVIDERCODE').merge(df_speciality, left_on='SPECIALTYCODE', right_on='SpecialtyCode')
# Select the relevant columns and drop duplicates to get unique combinations
df_segments = df_merged[['zip_code', 'FACILITYCODE', 'PROVIDERCODE', 'SpecialtyCode', 'SpecialtyDescription']].drop_duplicates()
# Display the unique combinations
df_segments = df_segments.rename(columns={'PROVIDERCODE': 'ProviderCode'})
df_segments.columns

Index(['zip_code', 'FACILITYCODE', 'ProviderCode', 'SpecialtyCode',
       'SpecialtyDescription'],
      dtype='object')

In [11]:
## load demographics ##
person_df = pd.read_csv(data_path + 'person_demographics.csv')
person_df = person_df[['PERSON_ID','age','GENDER']]
person_df['GENDER'] = person_df['GENDER'].apply(lambda x: 'Male' if pd.isnull(x) or x != 'Female' else 'Female')
person_df.columns

Index(['PERSON_ID', 'age', 'GENDER'], dtype='object')

In [12]:
person_df.GENDER.value_counts()

Male      7791
Female    2209
Name: GENDER, dtype: int64

In [13]:
provider_df = pd.read_csv(data_path + 'Provider/provider.csv')
#provider_df.columns

In [14]:
#included ratings
eligible_persons = list((set(df_ratings_score.PERSON_ID).intersection(set(person_df.PERSON_ID))))
df_ratings_fltrd = df_ratings_score[df_ratings_score['PERSON_ID'].isin(eligible_persons)]
df_ratings_fltrd.columns

Index(['PERSON_ID', 'ProviderCode', 'SurveyAnswerScore', 'RecommendStar',
       'ratings'],
      dtype='object')

In [15]:
#provider ranking 
df_provider_rankings = df_ratings_score.groupby(['ProviderCode']).agg({'SurveyAnswerScore' : 'mean','RecommendStar':'mean'}).reset_index()
df_provider_rankings['SurveyAnswerScore'] = np.round(df_provider_rankings['SurveyAnswerScore'],2)
df_provider_rankings['RecommendStar'] = np.round(df_provider_rankings['RecommendStar'],2)
df_provider_rankings = df_provider_rankings.rename(columns={'SurveyAnswerScore': 'survey_score' , 'RecommendStar' : 'star_rating'})

In [16]:
df_provider_rankings.columns

Index(['ProviderCode', 'survey_score', 'star_rating'], dtype='object')

In [17]:
## this is person demographics
person_df= person_df[person_df['PERSON_ID'].isin(eligible_persons)]
person_df = person_df.sort_values(by=['PERSON_ID', 'GENDER', 'age'], ascending=[True, True, False])
person_df = person_df.drop_duplicates(subset='PERSON_ID', keep='first')
person_df.head(1)

,PERSON_ID,age,GENDER
1190,00123db9a36e9f9277d6ce32c3fdd1e49809845399741b...,39.0,Female


In [18]:
ratings_df  = df_ratings_score
provider_rankings_df  = df_provider_rankings
segments_df  = df_segments
facility_df  = df_facility
provider_facility_df = df_provider_facility
provider_specialty_df = df_provider_speciality

In [19]:
interaction_df = pd.merge(ratings_df, person_df, on='PERSON_ID')
interaction_df = pd.merge(interaction_df, provider_rankings_df, on='ProviderCode')
interaction_df = pd.merge(interaction_df, segments_df, on='ProviderCode')

interaction_df = pd.get_dummies(interaction_df, columns=['GENDER'])

interaction_df = interaction_df.dropna()
interaction_df=interaction_df.drop_duplicates()
interaction_df.head(2)

,PERSON_ID,ProviderCode,SurveyAnswerScore,RecommendStar,ratings,age,survey_score,star_rating,zip_code,FACILITYCODE,SpecialtyCode,SpecialtyDescription,GENDER_Female,GENDER_Male
0,00123db9a36e9f9277d6ce32c3fdd1e49809845399741b...,XF6BG,2.500000,5.0,3.75,39.0,3.05,3.67,34950,81DB7D,PS305,Family Medicine,1,0
1,32e8750a2e31a34c71d700df32a773adcab8db3109117b...,XF6BG,4.708333,5.0,4.85,39.0,3.05,3.67,34950,81DB7D,PS305,Family Medicine,0,1


In [20]:
interaction_df.to_csv('metadata/interactions.csv')
provider_rankings_df.to_csv('metadata/provider_profile.csv')
segments_df.to_csv('metadata/zip-speciality-segments.csv')
facility_df.to_csv('metadata/facility-details.csv')
provider_facility_df.to_csv('metadata/provider-facility-map.csv')
provider_specialty_df.to_csv('metadata/provider-speciality-map.csv')
df_speciality.to_csv('metadata/spciality-details.csv')
person_df.to_csv('metadata/person-demographics.csv')

In [21]:
segments_df

,zip_code,FACILITYCODE,ProviderCode,SpecialtyCode,SpecialtyDescription
0,33328,100360,YVMCR,PS628,Orthopedic Spine Surgery
4,20900,6E1D9D,YVMCR,PS628,Orthopedic Spine Surgery
8,33328,100360,XYN6P6J,PS628,Orthopedic Spine Surgery
10,33029,9D6BB8,XYN6P6J,PS628,Orthopedic Spine Surgery
11,33133,C89B06,XYN6P6J,PS628,Orthopedic Spine Surgery
...,...,...,...,...,...
2427,33324,54F042,Y9MFT,PS250,"Endocrinology, Diabetes & Metabolism"
2428,34952,EA4BDF,XYMD45G,PS250,"Endocrinology, Diabetes & Metabolism"
2429,34952,EA4BDF,26GKH,PS250,"Endocrinology, Diabetes & Metabolism"
2430,33324,54F042,22F4V,PS328,Dentistry
